In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp
np.random.seed(42)

In [3]:
df = pd.read_csv("/Users/shiva/Downloads/real_time_fraud_data.csv")
df.info(memory_usage='deep')
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   transaction_id                10000 non-null  object 
 1   transaction_time              10000 non-null  object 
 2   user_id                       10000 non-null  object 
 3   merchant_id                   10000 non-null  object 
 4   amount                        10000 non-null  float64
 5   amount_to_avg_ratio           10000 non-null  float64
 6   txns_last_10min               10000 non-null  int64  
 7   avs_pass                      10000 non-null  int64  
 8   cvv_match                     10000 non-null  int64  
 9   category_risk_score           10000 non-null  float64
 10  days_since_account_creation   10000 non-null  int64  
 11  past_chargeback_flag          10000 non-null  int64  
 12  geo_distance_km               10000 non-null  float64
 13  is

(10000, 33)

In [4]:
df.describe()

,amount,amount_to_avg_ratio,txns_last_10min,avs_pass,cvv_match,category_risk_score,days_since_account_creation,past_chargeback_flag,geo_distance_km,is_proxy_ip,...,merchant_distinct_users_30d,merchant_fraud_rate_30d,merchant_amount_to_avg_ratio,user_txn_count_lifetime,user_fraud_count_lifetime,user_fraud_rate_lifetime,days_since_last_fraud,user_txn_count_90d,user_fraud_count_90d,user_fraud_rate_90d
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,33.513216,1.646645,1.004600,0.955600,0.981200,0.495083,1009.21650,0.100200,1995.880476,0.102100,...,1.806100,0.013663,1.876300,4.346400,0.090800,0.020700,928.705700,1.094900,0.021900,0.013792
std,43.816340,2.219301,0.992208,0.205992,0.135825,0.289366,573.43795,0.300281,1962.489331,0.302795,...,1.299643,0.086484,4.728772,1.856758,0.304244,0.077229,231.424274,1.050335,0.146364,0.102377
min,0.582188,0.015205,0.000000,0.000000,0.000000,0.000101,1.00000,0.000000,0.706303,0.000000,...,0.000000,0.000000,0.004327,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.186810,0.506349,0.000000,1.000000,1.000000,0.239769,520.00000,0.000000,568.909789,0.000000,...,1.000000,0.000000,0.358285,3.000000,0.000000,0.000000,999.000000,0.000000,0.000000,0.000000
50%,20.074927,0.994134,1.000000,1.000000,1.000000,0.499446,1011.00000,0.000000,1403.014396,0.000000,...,2.000000,0.000000,0.837160,4.000000,0.000000,0.000000,999.000000,1.000000,0.000000,0.000000
75%,39.608286,1.958969,2.000000,1.000000,1.000000,0.743742,1504.00000,0.000000,2779.123198,0.000000,...,3.000000,0.000000,1.809347,5.000000,0.000000,0.000000,999.000000,2.000000,0.000000,0.000000
max,903.898763,73.615645,8.000000,1.000000,1.000000,0.999992,1999.00000,1.000000,17093.059125,1.000000,...,8.000000,1.000000,202.652528,12.000000,2.000000,1.000000,999.000000,8.000000,1.000000,1.000000


In [5]:
df.head(5)

,transaction_id,transaction_time,user_id,merchant_id,amount,amount_to_avg_ratio,txns_last_10min,avs_pass,cvv_match,category_risk_score,...,merchant_distinct_users_30d,merchant_fraud_rate_30d,merchant_amount_to_avg_ratio,user_txn_count_lifetime,user_fraud_count_lifetime,user_fraud_rate_lifetime,days_since_last_fraud,user_txn_count_90d,user_fraud_count_90d,user_fraud_rate_90d
0,T00001,2025-03-01 09:45:31,U00861,M0247,6.661237,1.151164,3,1,1,0.672425,...,1.0,0.0,0.188454,2,0,0.0,999.0,1.0,0.0,0.0
1,T00002,2024-07-04 16:04:58,U01295,M0068,44.100826,1.973076,0,1,1,0.229698,...,2.0,0.0,1.568721,4,0,0.0,999.0,0.0,0.0,0.0
2,T00003,2024-09-25 01:53:46,U01131,M0445,29.120985,0.469709,1,1,1,0.322789,...,0.0,0.0,0.868940,3,0,0.0,999.0,1.0,0.0,0.0
3,T00004,2024-10-19 07:28:52,U01096,M0445,21.872934,0.501742,0,1,1,0.462080,...,0.0,0.0,0.652666,4,0,0.0,999.0,1.0,0.0,0.0
4,T00005,2025-01-13 05:55:17,U01639,M0326,16.795116,2.413396,2,1,1,0.589555,...,3.0,0.0,1.046056,5,0,0.0,999.0,0.0,0.0,0.0


In [6]:
##dataframe memory optimization

float_cols = df.select_dtypes(include=['float64']).columns
int_cols = df.select_dtypes(include=['int64']).columns

df[float_cols] = df[float_cols].astype('float32')
df[int_cols] = df[int_cols].astype('int32')

print("Memory optimization of column types:")
df.info(memory_usage='deep')

Memory optimization of column types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   transaction_id                10000 non-null  object 
 1   transaction_time              10000 non-null  object 
 2   user_id                       10000 non-null  object 
 3   merchant_id                   10000 non-null  object 
 4   amount                        10000 non-null  float32
 5   amount_to_avg_ratio           10000 non-null  float32
 6   txns_last_10min               10000 non-null  int32  
 7   avs_pass                      10000 non-null  int32  
 8   cvv_match                     10000 non-null  int32  
 9   category_risk_score           10000 non-null  float32
 10  days_since_account_creation   10000 non-null  int32  
 11  past_chargeback_flag          10000 non-null  int32  
 12  geo_distance_km         

In [7]:
# Fraction of fraud cases
fraud_ratio = df['fraud_flag'].mean()
print(f"fraud fraction: {fraud_ratio:.6f} ({fraud_ratio*100:.3f}% )")
df['fraud_flag'].value_counts()

fraud fraction: 0.020700 (2.070% )


fraud_flag
0    9793
1     207
Name: count, dtype: int64

In [8]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(sampling_strategy=0.5, random_state=42)
# X_res, y_res = sm.fit_resample(X, y)

missing_pct = df.isna().mean().sort_values(ascending=False)
print(missing_pct[missing_pct>0])

Series([], dtype: float64)


In [9]:
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols].nunique().sort_values(ascending=False)

transaction_id      10000
transaction_time     9998
user_id              2889
merchant_id           499
dtype: int64

In [10]:
# Class counts & ratios
counts = df['fraud_flag'].value_counts()
ratios = df['fraud_flag'].value_counts(normalize=True)
print(pd.concat([counts, ratios], axis=1, keys=['count','ratio']))

            count   ratio
fraud_flag               
0            9793  0.9793
1             207  0.0207


In [11]:
# Only select numeric columns for correlation
num_cols = df.select_dtypes(include=['number']).columns

# Compute correlations with 'is_fraud'
num_corr = df[num_cols].corr()['fraud_flag'].abs().sort_values(ascending=False)

print("Top numeric correlations with fraud_flag:")
print(num_corr.head(10))

Top numeric correlations with fraud_flag:
fraud_flag                   1.000000
user_fraud_rate_lifetime     0.542392
user_fraud_count_lifetime    0.476052
days_since_last_fraud        0.472301
user_fraud_rate_90d          0.245699
user_fraud_count_90d         0.237387
merchant_fraud_count_30d     0.055223
merchant_fraud_rate_30d      0.049041
hour_of_day_deviation        0.017771
txns_last_10min              0.017664
Name: fraud_flag, dtype: float64


In [12]:
cat_cols = df.select_dtypes(include=['object']).columns

for col in cat_cols:
    try:
        if df.groupby(col)['fraud_flag'].nunique().eq(1).all():
            print(f"⚠️ Potential leakage in '{col}' — perfect predictor")
    except Exception as e:
        print(f"Could not evaluate column {col}: {e}")

⚠️ Potential leakage in 'transaction_id' — perfect predictor
⚠️ Potential leakage in 'transaction_time' — perfect predictor


In [13]:
dup_count = df.duplicated().sum()
print(f"Duplicate rows: {dup_count}")
if dup_count > 0:
    df.drop_duplicates(inplace=True)
    print("Duplicates dropped")

Duplicate rows: 0


In [14]:
import pyarrow
df.to_parquet(r'/Users/shiva/Downloads/real_time_fraud_cleaned.parquet', index=False)
print("Cleaned data saved to 'real_time_fraud_cleaned.parquet'")

Cleaned data saved to 'real_time_fraud_cleaned.parquet'


In [15]:
##Feature engineering steps
df = pd.read_parquet('/Users/shiva/Downloads/real_time_fraud_cleaned.parquet')

In [16]:
##Encoding categorical variables
##transaction_id                10000 non-null  object 
##transaction_time              10000 non-null  object 
##user_id                       10000 non-null  object 
##merchant_id                   10000 non-null  object 

from sklearn.preprocessing import LabelEncoder

df.columns

# columns you want to label‑encode → new column name
ENCODE_MAP = {
    "transaction_id":         "transactionid",
    "transaction_time":        "transactiontime",
    "user_id":     "userid",
    "merchant_id": "merchantid"# map label → final target
}

encoders = {}                      # (optional) keep fitted encoders

for col, new_col in ENCODE_MAP.items():
    if col in df.columns:
        le = LabelEncoder()
        df[new_col] = le.fit_transform(df[col])
        encoders[col] = le          # store for inverse‑transform / inference
        df.drop(columns=[col], inplace=True)
df.head(5)

,amount,amount_to_avg_ratio,txns_last_10min,avs_pass,cvv_match,category_risk_score,days_since_account_creation,past_chargeback_flag,geo_distance_km,is_proxy_ip,...,user_fraud_count_lifetime,user_fraud_rate_lifetime,days_since_last_fraud,user_txn_count_90d,user_fraud_count_90d,user_fraud_rate_90d,transactionid,transactiontime,userid,merchantid
0,6.661237,1.151164,3,1,1,0.672425,840,1,1404.276123,0,...,0,0.0,999.0,1.0,0.0,0.0,0,7465,826,246
1,44.100826,1.973076,0,1,1,0.229698,1257,0,1547.996704,0,...,0,0.0,999.0,0.0,0.0,0.0,1,973,1247,67
2,29.120985,0.469709,1,1,1,0.322789,571,0,1245.826050,0,...,0,0.0,999.0,1.0,0.0,0.0,2,3120,1089,444
3,21.872934,0.501742,0,1,1,0.462080,579,0,1041.324097,0,...,0,0.0,999.0,1.0,0.0,0.0,3,3794,1054,444
4,16.795116,2.413396,2,1,1,0.589555,1441,0,496.694366,0,...,0,0.0,999.0,0.0,0.0,0.0,4,6170,1581,325


In [18]:
# df = pd.concat([df, pd.get_dummies(df['device_type'], prefix='cat', drop_first=True)], axis=1)
# df = pd.concat([df, pd.get_dummies(df['region'], prefix='cat', drop_first=True)], axis=1)
# df.drop(columns=['region','device_type'], inplace=True)
# df.head(5)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   amount                        10000 non-null  float32
 1   amount_to_avg_ratio           10000 non-null  float32
 2   txns_last_10min               10000 non-null  int32  
 3   avs_pass                      10000 non-null  int32  
 4   cvv_match                     10000 non-null  int32  
 5   category_risk_score           10000 non-null  float32
 6   days_since_account_creation   10000 non-null  int32  
 7   past_chargeback_flag          10000 non-null  int32  
 8   geo_distance_km               10000 non-null  float32
 9   is_proxy_ip                   10000 non-null  int32  
 10  new_device_flag               10000 non-null  int32  
 11  hour_of_day_deviation         10000 non-null  float32
 12  users_same_ip_last_1h         10000 non-null  int32  
 13  ip